In [ ]:
## yikes idk how you would do this anymore
# %load_ext autoreload
# %aimport processor, basket, coin
# %matplotlib inline

In [ ]:
%reload_ext autoreload

In [3]:
%autoreload 1
import sys

import pandas as pd
import numpy as np

import datetime
from datetime import timezone
import pytz

from pycoingecko import CoinGeckoAPI

from cointegridy.src.classes.processor import Processor
from cointegridy.src.classes.basket import Basket
from cointegridy.src.classes.coin import Coin
sys.path.pop(0)
# from bin import Statistics, Tests, Transforms, Utils
import matplotlib.pyplot as plt



ImportError: cannot import name 'DataLoader' from 'cointegridy.src.classes.data_loader' (/Users/jorgecanedo/Desktop/cointegridy/cointegridy/src/classes/data_loader.py)

In [ ]:
# coin_list = cg.get_coins_list()

# ids = [coin_data['id'] for coin_data in coin_list]

# id_to_data = {}
# for coin_data in coin_list:
#     id_to_data[coin_data.pop('id')] = coin_data
# ids = id_to_data.keys()

In [ ]:
# COIN_COLS = ['id', 'symbol', 'name']
# df_coin_names = pd.DataFrame(coin_list, columns=['name'])

# coin_market = cg.get_coins_markets(vs_currency=NAT_CURR)
# df_market = pd.DataFrame(coin_market, columns=['id', 'current_price', 'high_24h', 'low_24h'])
# df_market.set_index('id', inplace=True)

In [ ]:
###############
## CONSTANTS ##
###############

NAT_TZ = 'America/Los_Angeles'
NAT_TZOBJ = pytz.timezone(NAT_TZ)

TRG_TZOBJ = timezone.utc

PORT_IDS = ['bitcoin', 'bitcoin-cash', 'ethereum', 'litecoin']
start_date, end_date = (2020,1,1), (2021,10,31)

NAT_CURR = 'usd'

LOOKBACKS = [7, 14, 30]
ROLL_WIND = LOOKBACKS[1]

In [ ]:
##################
## LOCAL DRIVER ##
##################

cgp = CGProcessor()

# Create portfolio dataframe
port_val = cgp.create_portfolio(PORT_IDS, start_date, end_date)
data = cgp.data

cgp.normalize([], port=True)

In [ ]:
display(data)

# Tests
Test cointegration of coin basket: {coins} = {c_1,...,c_n}
ie. Check that the set {coins} is cointegrated using Engle-Granger two-step test
This means there is some STATIONARY linear combination of {coins}

<ol>
    <li>Check that c_i is order-1 integrable</li>
    <li>Create basket of coins, find linear regression.</li>
    <li>Verify spread of basket is cointegrated.</li>
    <li>Define strategy for basket trades.</li>
</ol>

#### Concerns
<ul>
    <li>Should we normalize data before running our regression?</li>
</ul>

#### Conclusions
<ul>
    <li>OLS and Linear Regression yield the same coefficients</li>
    <li>While we usually assume stock data is I(1), it sometimes isn't (eg. Litecoin, BTC-Cash)</li>
    <li>Normalization might help us find a regression line that does cointegrate the portfolio</li>
    <li>We should run this test on different coins in different sized baskets.</li>
</ul>

In [ ]:
############
## TEST 1 ##
############

# print("Normalizing...")
# scaler = MinMaxScaler()
# scaler.fit(X, y)
# X_scaled = scaler.transform(X) # TODO: Finish normalization
# X_scaled = pd.DataFrame(X_scaled/X_scaled[0], index=X.index) 
# X_scaled.columns = X.columns

print("Verifying coin price stationarity...")
coins = []
for coin_name in data:
    coin = Coin(coin_name)
    result = coin.is_good(data[coin_name], start_date, end_date)
    if result:
        coins.append(coin)
    else:
        print(coin_name, "is not I(1)")

print([coin.name_ for coin in coins], " are stationary \n")
        
basket = Basket(coins, target='bitcoin', processor=cgp)
basket.fit(data)
spread = basket.find_spread(data)
is_cointegrated = basket.is_coint(spread)
print("\n")
print("Cointegration test returned:", is_cointegrated)

short_entry, long_entry = basket.strat(spread)

In [ ]:
############
## TEST 2 ##
############

# Create normalized timeseries for portfolio
port_rm = cgp.roll_avg(ROLL_WIND)
port_rm.name = f'{port_val.name}_RM'


# IF a random variable is stationary THEN it is also order one integratable #

port_ret = Transforms.integrate(port_rm)

comparator = port_ret + np.random.normal(0, 5, len(port_ret))

# These two series should be stationary and cointegrate
# TODO: This needs to be standardized as a basket, then 
# coint_res = Tests.is_coint(port_ret, comparator)
#print(coint_res)

### Testing cointegration of crypto basket
Test cointegration of coin basket: {coins} = {c_1,...,c_n}
ie. Check that the set {coins} is cointegrated using Engle-Granger two-step test
This means there is some STATIONARY linear combination of {coins}

<ol>
    <li>Check that c_i is order-1 integrable</li>
    <li>Create basket of coins, find linear regression.</li>
    <li>Verify spread of basket is cointegrated.</li>
    <li>Define strategy for basket trades.</li>
</ol>

#### Concerns
<ul>
    <li>Should we normalize data before running our regression?</li>
</ul>

#### Conclusions
<ul>
    <li>OLS and Linear Regression yield the same coefficients</li>
    <li>While we usually assume stock data is I(1), it sometimes isn't (eg. Litecoin, BTC-Cash)</li>
    <li>Normalization might help us find a regression line that does cointegrate the portfolio</li>
    <li>We should run this test on different coins in different sized baskets.</li>
</ul>

In [ ]:
############
## TEST 3 ##
############

d_1, d_2 = CGProcessor.id_to_prices(cgp.cg, PORT_IDS[0], start_date, end_date), CGProcessor.id_to_prices(cgp.cg, PORT_IDS[1], start_date, end_date)
s_1, s_2 = pd.Series(d_1), pd.Series(d_2)
s_1.name, s_2.name = PORT_IDS[0], PORT_IDS[1]

# Random two series likely should not be stationary
coint_res = Tests.is_coint(s_1, s_2)
print(coint_res)

In [ ]:
cgp.take_mean(s_1)
df_feats = cgp.series_to_df_feats(s_1, LOOKBACKS, plot=True)

print(Statistics.sharpe_ratio(df_feats[f'{s_1.name}_DR']))


# Explore Exchanges on CoinGecko

In [ ]:
exchanges = cgp.cg.get_exchanges_list()
print("There are", len(exchanges), "exchanges.")

by_24hr_vol = sorted(exchanges, key=lambda d: d['trade_volume_24h_btc'], reverse=True) 
print("Top 3 by 24 trading volume:", [exchange['id'] for exchange in by_24hr_vol[:3]])
by_trust_score = sorted(exchanges, key=lambda d: d['trust_score_rank'])
print("Top 3 by trust score:", [exchange['id'] for exchange in by_trust_score[:3]])

For now just use the highest 100 market cap coins on coingecko

In [ ]:
COLS = ['id', 'current_price', 'high_24h', 'low_24h', 'market_cap', 'total_volume']

coin_market = cgp.cg.get_coins_markets(vs_currency=NAT_CURR)
df_market = pd.DataFrame(coin_market, columns=COLS)
df_market.set_index('id', inplace=True)

In [ ]:
df_market

From binomial theorem there are $2^{100}$ baskets so we need to find a way of filtering.

<b>There are also too many coins to pull all of them at once! We might have to create and host a database that contains updated data. We can pull around 50 or so coins at once.</b>

### Strategy
Take Pearson (Spearman?) correlation of dataframe to filter out some coins.

Idea:
We could train a model to take price data and try to spit out which coins are the most likely to be cointegrated. I think this could be some kind of 

In [ ]:
# Create portfolio dataframe
port_val = cgp.create_portfolio(df_market[:50].index, start_date, end_date)
data = cgp.data
# Write to CSV

In [ ]:
corr = data.corr()
valid = []


In [ ]:
# FOR GRAPHMASTER: What if we do what you said about constructing a graph with coins as 
# nodes and edges are pairwise correlation to find which coins are most important to
# test first? Could we use a GNN?


In [ ]:

print("Verifying coin price stationarity...")
coins = []
for coin_name in data:
    coin = Coin(coin_name)
    result = coin.is_good(data[coin_name], start_date, end_date)
    if result:
        coins.append(coin)
    else:
        print(coin_name, "is not I(1)")

print([coin.name_ for coin in coins], " are stationary \n")



In [ ]:
coinNames = [coin.name_ for coin in coins]

df = data[coinNames].corr().abs()

corrList = df.values.flatten()

In [ ]:
plt.hist(corrList,bins=40)

In [ ]:
df.values[tuple([np.arange(df.shape[0])])*2] = 0

In [ ]:
df

In [ ]:
plt.hist(df.values.flatten(),bins=40)

In [ ]:
def sigNormalizer(x):
    if x == 0:
        return 0
    
    return Utils.sigmoid((2*(x-1/2)))

My goal is to create a distribution with more extreme values and less good choices. Here's an example plot of what the sigmoid does:

In [ ]:
t = np.linspace(-10,10)

plt.plot(t,[sigNormalizer(k) for k in t])
print(Utils.sigmoid(10))

In [ ]:
df2 = df.copy()
display(df2)
df2 = df2.applymap(sigNormalizer)
#for c in df2.keys():
    #df2[c].map(lambda x: np.log(Utils.sigmoid(x+10)))

In [ ]:
df2

In [ ]:
plt.hist(df2.values.flatten(),bins=40)

This is a better-looking distribution. This will come in handy later when we construct a graph.

# The graph approach to coint search

Construct a graph with coins as nodes and weighted edges representing correlation. The adjacency matrix will be df2 (activation function on absolute value of correlation).


In [ ]:
# Find coins

coinDict = {x.name_:i for i,x in enumerate(coins)}

In [ ]:
coinDict

In [ ]:
Utils.softmax(np.array([2,3,8,4]))

In [ ]:
# Greedy approach: on the graph represented by the adj matrix DF2, we go to a node, find the most related node by Pearson corr, and repeat for basket size. 

def find_baskets_determ(size,numToFind):
    coinList = coins.copy
    baskets = []
    temp = df2.copy()
    firsts = {} # This is an array to remove redundancy.  
    for _ in range(numToFind):
        
        bask = []
        node = np.random.choice(coins).name_
        for _ in range(size):
            bask.append(node)
            canGo = df[node].drop(labels=bask)
            bestNeighbor = np.random.choice(canGo.index,p=Utils.softmax(df[node].drop(labels=bask).to_numpy()))
            
            #Utils.softmax(df[node].drop(labels=bask).to_numpy())
            #.idxmax()
            node = bestNeighbor
        baskets.append(bask)
    return list(set(baskets))
            #temp.drop(node.name_,axis=0)
            #temp.drop(node.name_,axis=1)



In [ ]:
def find_baskets_determ(size,numToFind):
    coinList = coins.copy
    baskets = []
    temp = df2.copy()
    firsts = {} # This is an array to remove redundancy.  
    for _ in range(numToFind):
        
        bask = []
        node = np.random.choice(coins).name_
        for _ in range(size):
            bask.append(node)
            bestNeighbor = df[node].drop(labels=bask).idxmax()
            node = bestNeighbor
        baskets.append(tuple(bask))
    return list(set(baskets))
            #temp.drop(node.name_,axis=0)
            #temp.drop(node.name_,axis=1)

In [ ]:
found_baskets = find_baskets_determ(5,60)
found_baskets

In [ ]:
master_list = []
for bk in found_baskets:
    bask = [coins[coinDict[x]] for x in bk]
    print(bask)
    basket = Basket(bask, target=bask[0].name_, processor=cgp)
    print(f'Testing basket: {bask}')
    basket.fit(data)
    spread = basket.find_spread(data)
    is_cointegrated = basket.is_coint(spread)
    print("\n")
    print("Cointegration test returned:", is_cointegrated)
    master_list.append([basket,is_cointegrated])

In [ ]:
master_list

In [ ]:
basket,isc = master_list[20]
print([x.name_ for x in basket.coins_])
short_entry, long_entry = basket.strat(spread)

In [ ]:
data[coinNames]

In [ ]:
Statistics.sharpe_ratio(data['cdai']/data.iloc[0,'cdai'])

In [ ]:
data[coinNames]/(data.iloc[0,:]+0.00001)

In [ ]:
data['binance-usd']